# Tratamiento de datos

En este notebook se realizará la extracción y el respectivo tratamiento de los dataframes que se usarán en el análisis. Los dataframes que se generarán serán los siguientes
 * [info_countries.csv](data/info_countries.csv): Información variada sobre los países en español.
 * [olympics.csv](data/olympics.csv): Dataframe con la información de las medallas ganadas por equipo olímpico.
El dataframe `olympics.csv` será la base de datos principal para este análisis.

Adicionalmente, también se ha generado un archivo XLSX llamado [olympics_excel.xlsx](data/olympics_excel.xlsx) con información idéntica a la que contiene `olympics.csv`.

Estos datos se almacenarán en el directorio,

In [1]:
data_path = "data/"

## Imports

Aquí se presentan los paquetes que se usarán para el tratamiento de datos.

In [2]:
import pandas as pd
import numpy as np

In [3]:
import re
import os

from unidecode import unidecode
from countryinfo import CountryInfo
from googletrans import Translator
import pycountry

## Fuentes

A continuación se listan las url donde se obtienen los datos

 * Medallas ganadas por país

In [4]:
url_medallas = "https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"

 * Lista de países por continente

In [5]:
url_paises = "https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses_por_continentes"

 * Equivalencias entre códigos DOI y ISO

In [6]:
url_codigos = "https://es.wikipedia.org/wiki/Anexo:Comparaci%C3%B3n_de_los_c%C3%B3digos_del_COI,_la_FIFA_y_la_ISO_3166"

 * Parámetros e indicadores de los países.
   * World Bank Group DataBase: `https://data.worldbank.org/`

## Extracción de los dataframes

### Dataframe de medallas por país

A continuación se extrae la información que Wikipedia posee de las medallas olímpicas ganadas por país, tanto en olimpiadas de verano como en las de invierno.

Primero extraemos de la url correspondiente.

In [7]:
df = pd.read_html(url_medallas)[1]

Renombramos las columnas para que la manipulación del data frame sea más cómodo

In [8]:
columns = ['Country', 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']

df.columns = columns
df = df[df["Country"] != "Totals"] # Eliminados fila de totales
df.head()

,Country,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
0,Afghanistan (AFG),16,0,0,2,2,0,0,0,0,0,16,0,0,2,2
1,Albania (ALB),10,0,0,2,2,5,0,0,0,0,15,0,0,2,2
2,Algeria (ALG),15,7,4,9,20,3,0,0,0,0,18,7,4,9,20
3,Argentina (ARG),26,22,27,31,80,20,0,0,0,0,46,22,27,31,80
4,Armenia (ARM),8,2,11,9,22,8,0,0,0,0,16,2,11,9,22


Una columna posee tanto el nombre del país como el código COI, por lo que crearemos dos columnas más donde los tratemos de forma separada

In [9]:
df["Country_code_coi"] = df.Country.str.extract(r"\((\w{3})\)")[0]
df["Country_name"] = df.Country.str.extract(r"(.*)\s\(")[0]
df["Country"] = df["Country_name"] + " (" + df["Country_code_coi"] + ")"
df = df[['Country', 'Country_name','Country_code_coi', 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']] # Ordenamos las columnas
df.head()

,Country,Country_name,Country_code_coi,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
0,Afghanistan (AFG),Afghanistan,AFG,16,0,0,2,2,0,0,0,0,0,16,0,0,2,2
1,Albania (ALB),Albania,ALB,10,0,0,2,2,5,0,0,0,0,15,0,0,2,2
2,Algeria (ALG),Algeria,ALG,15,7,4,9,20,3,0,0,0,0,18,7,4,9,20
3,Argentina (ARG),Argentina,ARG,26,22,27,31,80,20,0,0,0,0,46,22,27,31,80
4,Armenia (ARM),Armenia,ARM,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22


Las columnas del dataframe están listas y son las siguientes
 * `Country`: Nombre del país y código expedido por el Comité Olímpico Internacional.
 * `Country_name`: Nombre del país.
 * `Country_code_coi`: Código expedido por el Comité Olímpico Internacional.
 * `Num_Summer`: Número de participaciones en Olimpiadas de Verano.
 * `gold_summer`: Número de medallas de oro ganadas en Olimpiadas de Verano.
 * `silver_summer`: Número de medallas de plata ganadas en Olimpiadas de Verano.
 * `bronze_summer`: Número de medallas de bronce ganadas en Olimpiadas de Verano.
 * `Total_summer`: Total de medallas de ganadas en Olimpiadas de Verano.
 * `Num_Winter`: Número de participaciones en Olimpiadas de Invierno.
 * `gold_winter`: Número de medallas de oro ganadas en Olimpiadas de Invierno.
 * `silver_winter`: Número de medallas de plata ganadas en Olimpiadas de Invierno.
 * `bronze_winter`: Número de medallas de bronce ganadas en Olimpiadas de Invierno.
 * `Total_winter`: Total de medallas de ganadas en Olimpiadas de Invierno.
 * `Num_Games`: Total de participaciones.
 * `gold`: Total de medallas de oro ganadas.
 * `silver`: Total de medallas de plata ganadas.
 * `bronze`: Total de medallas de bronce ganadas.
 * `Combined_total`: Total de medallas combinadas.

Y sus dimensiones son,

In [10]:
df.shape

(162, 18)

### Dataframe de países

Con la finalidad de conocer un poco mejor a los países, vamos a importar un dataframe con un poco de su información.

Primero vamos con la clasificación de países por continente. Para ello se extraerá la información de un articulo de la Wikipedia en Español llamada *Anexo:Países por continentes*, que clasifica en varias tablas, cada uno representando un continente. Dada la complejidad de esta extracción, crearemos algunas funciones que permitirá hacer la extracción y el tratamiento correspondiente:

In [11]:
def tratamiento_columnas(dataframe):
	'''Deja las columnas sin espacios, con un guion bajo en su lugar, quita signos de puntuación,
		y hace coincidir de tablas distintas, columnas idénticas con diferentes nombres'''
	columns = dataframe.columns

	new_columns = []
	for col in columns:

		if not isinstance(col, str):
			new_columns.append(col)
			continue

		col = unidecode(col.lower()).strip()
		while "  " in col:
			col = col.replace("  ", " ")
		col = re.sub(r"\s", "_", col)

		if "superficie" in col:
			col = "superficie_km2"
		elif "poblacion" in col:
			col = "poblacion"
		elif "iso" in col:
			col = col.replace("iso", "ISO")
		elif "idh" in col:
			col = col.upper()
		elif col in ["nombre_oficial", "pais", "dependencia", "region_ultraperiferica", "region"]:
			col = "nombre"

		new_columns.append(col)

	dataframe.columns = new_columns

	return dataframe

def validar_continente(pais, series):
	''' Función auxiliar: revisa si un pais está en una series '''
	return any(map(lambda x: bool(re.search(pais, x)), series.values))

def titulos_tratamiento(string):
	'''Elimina rastros de las referencias de Wikipedia, y otras aclaraciones escritas entre paréntesis,
		o corchetes'''
	pattern = re.compile(r"(.*?)(?:\[.+\])+.*(\(.+\))?")

	if (match_ := pattern.fullmatch(string)):
		return match_.group(1)
	else:
		return string

def data_frame_paises():
	'''Extrae datos de los países de una página que tabula los países por continente. Usando un mapeo con
		una muestra de cada tabla, es capaz de clasificar agregar una columna etiquetando cada país con
		el continente correcto'''

	mapping = {
		"Alemania" : "Europa",
		"Kosovo" : "Europa", # Estados con reconocimiento limitado
		"China" : "Asia",
		"Israel" : "Asia",  # Estados con reconocimiento limitado
		"Isla de Navidad" : "Asia", # Territorios dependientes
		"Macao": "Asia", # Regiones administrativas especiales de la República Popular de China
		"Angola" : "África",
		"Chile" : "América",
		"Anguila" : "América", # Territorios dependientes
		"Guadalupe" : "América", # Países americanos integrados en estados no americanos
		"Australia" : "Oceanía",
		"Guam" : "Oceanía" # Territorios dependientes
	}

	df_continents = pd.read_html(url_paises)

	tablas = []

	df_tratados = map(tratamiento_columnas, df_continents)

	for tb in df_tratados:
		try:
			map_country = list(filter(lambda x: validar_continente(x, tb["nombre"]), mapping.keys()))[0]
		except (KeyError, IndexError) as e:
			#print(e)
			continue

		tb["Continente"] = mapping[map_country]
		tablas.append(tb.copy())

	df_countries = pd.concat(tablas, axis=0)
	df_countries = df_countries.reset_index()
	df_countries["nombre"] = df_countries["nombre"].map(titulos_tratamiento)

	return df_countries

def extraer_codigo_iso(series):
	'''Extraer el código ISO 3166-1 alpha 3 de los países en base al nombre del país en español'''

	traducidos = {
		'Botsuana': 'Botswana',
		'Comoras': 'Comoros',
		'Gambia': 'Republic of The Gambia',
		'Guinea-Bisáu': 'Guinea-Bissau',
		'Malaui': 'Malawi',
		'Mauricio': 'Mauritius',
		'República Democrática del Congo': 'Democratic Republic of the Congo',
		'República del Congo': 'Republic of the Congo',
		'Santo Tomé y Príncipe': 'São Tomé and Príncipe',
		'Somalilandia (República de Somalilandia)': 'Somaliland',
		'Suazilandia': 'Swaziland',  # Suazilandia cambió su nombre a Eswatini en 2018
		'Sudán del Sur (República de Sudán del Sur)': 'South Sudan',
		'Yibuti': 'Djibouti'
	}

	if isinstance(series["ISO_code"], str):
		if len(series["ISO_code"]) == 3:
			return series["ISO_code"]

	t = Translator()
	pais_ = series.nombre

	pais_ingles = t.translate(pais_, src="es", dest="en")
	countryinfo = CountryInfo(pais_ingles.text)

	try:
		return countryinfo.iso(3)
	except KeyError:
		try:
			countryinfo = CountryInfo(traducidos[pais_])
			return countryinfo.iso(3)
		except KeyError:
			#print(f"No se pudo obtener código ISO de {pais_}")
			return np.nan

Creadas las funciones procedamos con la extracción de la información:

In [12]:
# Extraer información
df_countries = data_frame_paises()

# Eliminar columnas nulas
df_countries = df_countries.dropna(axis=1, how="all")

# Tratando la columna con los códigos ISO
df_countries = df_countries.rename(columns={"codigo_ISO_3166-1" : "ISO_code"})
df_countries["ISO_code"] = df_countries.apply(extraer_codigo_iso, axis=1)
df_countries[df_countries["ISO_code"].isna()].to_csv(os.path.join(data_path, "info_countries_ISO_nan.csv"), index=False, encoding="utf-8")
df_countries = df_countries.dropna(subset="ISO_code")

# Seleccionando las columnas de interés y ordenándolas
df_countries = df_countries[
	['nombre', 'ISO_code', 'Continente']
]
df_countries.head()

,nombre,ISO_code,Continente
0,Albania,ALB,Europa
1,Alemania,DEU,Europa
2,Andorra,AND,Europa
3,Armenia,ARM,Europa
4,Austria,AUT,Europa


Las dimensiones del dataframe son:

In [13]:
df_countries.shape

(229, 3)

Cabe destacar que el articulo de Wikipedia de donde estamos extrayendo la información del dataframe `df_countries` posee mucha mas información de la que conservamos. La razón es porque de esta fuente solo nos interesa obtener la clasificación por Continente que proporciona, así como el código ISO que es único por país.

#### Países transcontinentales

Dado que existen países que abarcan varios continentes, esto se traduce como la existencia de registros casi duplicados en `df_countries`. Estos registros son idénticos, salvo que difieren en el continente, lo que pone en evidencia su estatus de países transcontinentales. A continuación se listan aquellos que cumplen este criterio,

In [14]:
df_countries[df_countries.duplicated(subset=["nombre"], keep=False)].sort_values(by=["nombre"])

,nombre,ISO_code,Continente
106,Abjasia,ABH,Asia
50,Abjasia,ABH,Europa
3,Armenia,ARM,Europa
60,Armenia,ARM,Asia
61,Azerbaiyán,AZE,Asia
5,Azerbaiyán,AZE,Europa
10,Chipre,CYP,Europa
70,Chipre,CYP,Asia
131,Egipto,EGY,África
73,Egipto,EGY,Asia


Los países que se muestran en la lista son aquellos países que obtuvieron una clasificación del doble dada su condición de País Transcontinental. Para lograr una clasificación eficaz se recurrirá a relaciones culturales con cada continente donde su territorio se asienta. La elección hecha será la siguiente,
 * **Abjasia**: *Europa*; Aunque geográficamente se sitúa en Asia, tiene una relación más estrecha con Europa.
 * **Armenia**: *Europa*,
 * **Azerbaiyán**: *Asia*,
 * **Chipre**: *Europa*; Aunque geográficamente la mayor parte de su territorio está en Asia, tiene una relación más estrecha con Europa.
 * **Egipto**: *África*,
 * **Georgia**: *Europa*; La disparidad que se produjo al categorizarla en continente se debió a la ambigüedad de su posición geográfica, sin embargo, es más afín a Europa que a Asia.
 * **Isla de Navidad**: *Asía*; Aunque depende de Australia, su población es mayormente asiática.
 * **Kazajistán**: *Asia*,
 * **Osetia del Sur**: *Europa*; Aunque geográficamente se sitúa en Asia, tiene una relación más estrecha con Europa.
 * **Rusia**: *Europa*; A pesar de que la mayor parte de su territorio se encuentra en Asia, su centro de poder y su influencia cultural y política se encuentra en Europa.
 * **Turquía**: *Europa*; De la misma forma que Russia, Turkey esta más alineada cultural y políticamente a Europa.

Dado lo anterior, es necesario eliminar los registros que no son de interés, para esto crearemos un mapeo

In [15]:
map_eliminar_registros_casi_duplicados = {
	"Abjasia" : "Europa",
	"Armenia" : "Europa",
	"Azerbaiyán" : "Asia",
	"Chipre" : "Europa",
	"Egipto" : "África",
	"Georgia" : "Europa",
	"Isla de Navidad" : "Europa",
	"Kazajistán" : "Asia",
	"Osetia del Sur" : "Asia",
	"Rusia" : "Europa",
	"Turquía" : "Europa"
}

Y con este mapeo, eliminamos el registro que no es de interés,

In [16]:
index_eliminar = df_countries[df_countries.apply(lambda x: x["Continente"] != map_eliminar_registros_casi_duplicados[x["nombre"]] if x["nombre"] in map_eliminar_registros_casi_duplicados else False, axis=1)].index
df_countries = df_countries.drop(index_eliminar)

#### Errores con el código ISO.

Como ya hemos mencionado antes, el código ISO 3166-1 alpha 3 debe ser único para cada país. Tener dos o más países con el mismo código ISO representaría un problema, ya que este campo será la conexión con otras tablas que extraeremos. Por eso, vamos a verificar que no existan problemas de este tipo,

In [17]:
df_countries.groupby(["ISO_code"]).filter(lambda x: x["nombre"].nunique() > 1)

,nombre,ISO_code,Continente
99,Tayikistán,TJK,Asia
101,Turkmenistán,TJK,Asia


Para el día 8 de septiembre del 2024, al parecer existe un error donde Tayiskistán y Turkmenistán tienen el mismo ISO, TJK. Este código, según la ISO 31166-1, debería pertenecerle solo a Tayiskistán. El código ISO de Turkmenistán es TKM, vamos a editar ese registro

In [18]:
df_countries.loc[df_countries.nombre == "Turkmenistán", "ISO_code"] = "TKM"

#### Datos e Indicadores del Banco Mundial

Ahora, vamos a agregar más datos variados al dataframe, todos ellos extraídos del Banco Mundial (Word Bank Group). Con esto incluiremos bastante información relevante de cada país que nos permitirá realizar un análisis estadístico más completo. Para no saturar demasiado este notebook, se ha delegado el tratamiento de datos al notebook [Anexo_Tratamiento_datos_WBG.ipynb](Anexo_Tratamiento_datos_WBG.ipynb).

Resumiendo, se ha descargado una serie de archivos CSV en la carpeta `data/WBG_data/` referente a algunos indicadores y datos varios de los países, y los datos se han tratado y exportado en un único archivo CSV llamado [wbg_data.cvs](data/wbg_data.cvs) alojado en el directorio `data/`. vamos a extraerlo, 

In [19]:
df_wbg = pd.read_csv("data/wbg_data.cvs")
df_wbg.head()

,Country_Code,Short_Name,Table_Name,Long_Name,2-alpha_code,Currency_Unit,Region,WB-2_code,Access_to_electricity_media,AFFVA_media,Control_of_Corruption_media,Expenditure_on_education_media,GDP_per_capita_media,GDP_per_capitaPPP_media,population_media,surface_area_media
0,ABW,Aruba,Aruba,Aruba,AW,Aruban florin,Latin America & Caribbean,AW,99.472445,0.103728,1.139387,5.325764,21735.275013,33223.072570,7.790650e+04,1.800000e+02
1,AFE,Africa Eastern and Southern,Africa Eastern and Southern,Africa Eastern and Southern,ZH,NaN,NaN,ZH,31.756021,13.177775,NaN,4.410762,822.978192,2830.148059,3.579699e+08,1.512053e+07
2,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,South Asia,AF,57.091304,27.952003,-1.420090,2.956045,416.667609,1715.017152,1.878244e+07,6.528600e+05
3,AFW,Africa Western and Central,Africa Western and Central,Africa Western and Central,ZI,NaN,NaN,ZI,41.742255,22.446226,NaN,2.843056,881.552220,3104.291490,2.438662e+08,9.166270e+06
4,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,Sub-Saharan Africa,AO,36.286957,7.883305,-1.233098,3.034522,2133.920034,5274.217573,1.518896e+07,1.246700e+06


Y sus dimensiones son,

In [20]:
df_wbg.shape

(265, 16)

La finalidad del dataframe `df_wbg` es unirlo al dataframe `df_countries` para obtener información más completa de los países. La unión se hace mediante el código ISO,

In [21]:
df_countries = df_countries.merge(df_wbg, how="outer", left_on="ISO_code", right_on="Country_Code")

Ahora, dado que el tipo de unión que se realizó —Full Join—, habrá registros donde `ISO_code` y `Country_Code` tendrán valores nulos. Esto es bastante obvio, ya que habrá registros en un dataframe que no existan en otros. Para evitar problemas más adelante, vamos a rellenar los valores nulos de una columna con la otra,

In [22]:
df_countries["Country_Code"] = df_countries.apply(lambda x: x["ISO_code"] if pd.isna(x["Country_Code"]) else x["Country_Code"], axis=1)
df_countries["ISO_code"] = df_countries.apply(lambda x: x["Country_Code"] if pd.isna(x["ISO_code"]) else x["ISO_code"], axis=1)

Y revisamos si quedaron registros con valores nulos en ambas columnas,

In [23]:
df_countries[df_countries.ISO_code.isna() & df_countries.Country_Code.isna()].shape

(0, 19)

Por el momento el dataframe queda como

In [24]:
df_countries.head()

,nombre,ISO_code,Continente,Country_Code,Short_Name,Table_Name,Long_Name,2-alpha_code,Currency_Unit,Region,WB-2_code,Access_to_electricity_media,AFFVA_media,Control_of_Corruption_media,Expenditure_on_education_media,GDP_per_capita_media,GDP_per_capitaPPP_media,population_media,surface_area_media
0,Albania,ALB,Europa,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,Europe & Central Asia,AL,99.809091,20.936392,-0.672443,3.371380,2704.306845,8058.206926,2.713609e+06,28750.000000
1,Alemania,DEU,Europa,DEU,Germany,Germany,Federal Republic of Germany,DE,Euro,Europe & Central Asia,DE,100.000000,0.872843,1.819941,4.662717,22790.195498,37965.625121,7.980769e+07,357081.147541
2,Andorra,AND,Europa,AND,Andorra,Andorra,Principality of Andorra,AD,Euro,Europe & Central Asia,AD,100.000000,0.479975,1.252925,2.727460,25049.185525,42867.069216,5.048064e+04,470.000000
3,Armenia,ARM,Europa,ARM,Armenia,Armenia,Republic of Armenia,AM,Armenian dram,Europe & Central Asia,AM,99.521739,14.163111,-0.555450,2.667407,2545.232810,7498.401279,2.927306e+06,29740.000000
4,Austria,AUT,Europa,AUT,Austria,Austria,Republic of Austria,AT,Euro,Europe & Central Asia,AT,100.000000,2.223131,1.648681,5.246932,23702.201759,39846.724625,7.917847e+06,83879.000000


con las dimensiones

In [25]:
df_countries.shape

(278, 19)

#### Relación entre el código ISO, COI y FIFA

Dado que el código ISO y el código dado por el COI en ocasiones no coinciden, es necesario tener la comparación entre las dos. El articulo de la Wikipedia en Español titulado *Anexo:Comparación de los códigos del COI, la FIFA y la ISO 3166* tiene la relación que necesitamos. Adicionalmente, también cuenta con el código de la FIFA, que podría ser útil en algunos otros proyectos.

Obtengamos la información de la relación entre los códigos,

In [26]:
df_codigos = pd.read_html(url_codigos)[1]
df_codigos.columns = [
	"nombre", "COI", "FIFA", "ISO", "Observaciones"
]
df_codigos

,nombre,COI,FIFA,ISO,Observaciones
0,Afganistán,AFG,AFG,AFG,NaN
1,Åland,NaN,NaN,ALA,No es un país. Pertenece a Finlandia.
2,Albania,ALB,ALB,ALB,NaN
3,Alemania,GER,GER,DEU,NaN
4,Andorra,AND,AND,AND,NaN
...,...,...,...,...,...
249,Yemen,YEM,YEM,YEM,NaN
250,Yibuti,DJI,DJI,DJI,NaN
251,Wallis y Futuna,NaN,NaN,WLF,No es un país. Pertenece a Francia.
252,Zambia,ZAM,ZAM,ZMB,NaN


No todos los registros en `df_codigos` tienen un código ISO, ya que no todos son países. Algunos son equipos olímpicos, otros países que ya no existen —anteriores al código—, y otros son países con reconocimiento limitado. Sin embargo, es necesario conservar solo los registros con ISO no nulo.

Antes de comenzar la limpieza de los datos con código ISO nulo, en necesario mencionar a Kosovo. Este es uno de los registros con ISO nulo, como se muestra a continuación,

In [27]:
df_codigos[df_codigos.ISO.isna()]

,nombre,COI,FIFA,ISO,Observaciones
63,Escocia,NaN,SCO,NaN,Pertenece al Reino Unido. En competiciones de ...
76,Gales,NaN,WAL,NaN,Pertenece al Reino Unido. En competiciones de ...
99,Inglaterra,NaN,ENG,NaN,Pertenece al Reino Unido. En competiciones de ...
103,Irlanda del Norte,NaN,NIR,NaN,Pertenece al Reino Unido. En competiciones de ...
134,Kosovo,KOS,KVX,NaN,"No reconocido por la ONU, por lo que no aparec..."


donde lo acompañan las subdivisiones del Reino Unido. Sin embargo, la situación de Kosovo es diferente, ya que este es un estado con reconocimiento limitado y no tiene un código ISO oficial. No obstante, si que posee uno temporal: XKX.

Vamos a agregarlo y a limpiar los otros registros con código ISO nulo,

In [28]:
df_codigos.loc[df_codigos.nombre == "Kosovo", "ISO"] = "XKX"
df_codigos = df_codigos.dropna(subset=["ISO"])

Con esto el dataframe de códigos está listo para unirse al de países,

In [29]:
df_countries = pd.merge(df_countries, df_codigos[["COI", "FIFA", "ISO"]], how="left", left_on="Country_Code", right_on="ISO").drop(columns=["ISO"])

df_countries = df_countries[
	['nombre', 'Country_Code', 'COI', 'FIFA', 'Continente',
	'Short_Name', 'Table_Name', 'Long_Name',
	'2-alpha_code', 'Currency_Unit', 'Region', 'WB-2_code',
	'Access_to_electricity_media', 'AFFVA_media', 'Control_of_Corruption_media',
	'Expenditure_on_education_media','GDP_per_capita_media',
	'GDP_per_capitaPPP_media', 'population_media', 'surface_area_media']
]
df_countries = df_countries.rename(columns={"Country_Code" : "ISO_code"})
df_countries.head()

,nombre,ISO_code,COI,FIFA,Continente,Short_Name,Table_Name,Long_Name,2-alpha_code,Currency_Unit,Region,WB-2_code,Access_to_electricity_media,AFFVA_media,Control_of_Corruption_media,Expenditure_on_education_media,GDP_per_capita_media,GDP_per_capitaPPP_media,population_media,surface_area_media
0,Albania,ALB,ALB,ALB,Europa,Albania,Albania,Republic of Albania,AL,Albanian lek,Europe & Central Asia,AL,99.809091,20.936392,-0.672443,3.371380,2704.306845,8058.206926,2.713609e+06,28750.000000
1,Alemania,DEU,GER,GER,Europa,Germany,Germany,Federal Republic of Germany,DE,Euro,Europe & Central Asia,DE,100.000000,0.872843,1.819941,4.662717,22790.195498,37965.625121,7.980769e+07,357081.147541
2,Andorra,AND,AND,AND,Europa,Andorra,Andorra,Principality of Andorra,AD,Euro,Europe & Central Asia,AD,100.000000,0.479975,1.252925,2.727460,25049.185525,42867.069216,5.048064e+04,470.000000
3,Armenia,ARM,ARM,ARM,Europa,Armenia,Armenia,Republic of Armenia,AM,Armenian dram,Europe & Central Asia,AM,99.521739,14.163111,-0.555450,2.667407,2545.232810,7498.401279,2.927306e+06,29740.000000
4,Austria,AUT,AUT,AUT,Europa,Austria,Austria,Republic of Austria,AT,Euro,Europe & Central Asia,AT,100.000000,2.223131,1.648681,5.246932,23702.201759,39846.724625,7.917847e+06,83879.000000


El dataframe de paises queda con las siguientes columnas: 

 * `nombre`: (`str`) Nombre en español del país.
 * `ISO_code`: (`str`) Código ISO 3166-1 alpha-3, expedido al país por la Organización Internacional de Estandarización (ISO *siglas en inglés*).
 * `COI`: (`str`) Código del país expedido por el Comité Olímpico Internacional (COI)
 * `FIFA`: (`str`) Código del país expedido por el Federación Internacional de Fútbol Asociación (FIFA *siglas en francés*).
 * `Continente`: (`str`) Continente en español al que pertenece o perteneció el país: África, América, Asia, Europa u Oceanía.
 * `Short_Name`: (`str`) Nombre corto y abreviado del nombre del país en inglés. Ejemplo: Mexico.
 * `Table_Name`: (`str`) Nombre corto y abreviado del nombre del país en inglés para formato tabular. Puede o no coincidir con Short_Name.
 * `Long_Name`: (`str`) Nombre completo oficial en inglés del país. Ejemplo: United Mexican States.
 * `2-alpha_code`: (`str`) Código ISO 3166-1 alpha-2, expedido al país por la Organización Internacional de Estandarización (ISO *siglas en inglés*).
 * `Currency_Unit`: (`str`) Moneda de curso legal en el país. Nombre en inglés.
 * `Region`: (`str`) Clasificación regional hecho por el World Bank Group siguiendo criterios geográficos, económicos y, en algunos casos, políticos.
 * `WB-2_code`: (`str`) Código de dos letras que se usa para identificar países en los sistemas del World Bank Group.
 * `Access_to_electricity_media`: (`float`) Media histórica del indicador desde 1990 hasta 2022:
   * **Nombre del indicador en inglés**: *Access to electricity (% of population)*
   * **Código del indicador**: EG.ELC.ACCS.ZS
   * **Descripción**: Porcentaje de la población con acceso a la electricidad. Los datos sobre se recopilan de la industria, encuestas nacionales y fuentes internacionales.
 * `AFFVA_media`: (`float`) Media histórica del indicador desde 1960 hasta 2023:
   * **Nombre del indicador en inglés**: *Agriculture, forestry, and fishing, value added (% of GDP)*.
   * **Código del indicador**: NV.AGR.TOTL.ZS
   * **Descripción**: Mide la contribución porcentual del sector agrícola, silvicultural y pesquero al Producto Interno Bruto (PIB o GDP por sus siglas en inglés) en porcentaje.
 * `Control_of_Corruption_media`: (`float`) Media histórica del indicador desde 1996 hasta 2022:
   * **Nombre del indicador en inglés**: *Control of Corruption: Estimate*.
   * **Código del indicador**: CC.EST
   * **Descripción**: Captura percepciones de hasta qué punto el poder público se ejerce para beneficio privado, incluidas formas tanto pequeñas como grandes de corrupción, así como la "captura" del Estado por parte de élites e intereses privados. La estimación proporciona la puntuación del país en el indicador agregado, en unidades de una distribución normal estándar, es decir, entre aproximadamente $-2.5$ y $2.5$.
 * `Expenditure_on_education_media`: (`float`) Media histórica del indicador desde 1970 hasta 2023:
   * **Nombre del indicador en inglés**: *Government expenditure on education, total (% of GDP)*.
   * **Código del indicador**: SE.XPD.TOTL.GD.ZS
   * **Descripción**: El gasto del gobierno general en educación (corriente, capital y transferencias) se expresa como porcentaje del PIB. Incluye gastos financiados por transferencias de fuentes internacionales al gobierno. El gobierno general generalmente se refiere a los gobiernos locales, regionales y centrales.
 * `GDP_per_capita_media`: (`float`) Media histórica del indicador desde 1960 hasta 2023:
   * **Nombre del indicador en inglés**: *GDP per capita (current US$)*.
   * **Código del indicador**: NY.GDP.PCAP.CD
   * **Descripción**: El PIB per cápita es el producto interno bruto dividido por la población a mitad de año. El PIB es la suma del valor agregado bruto de todos los productores residentes en la economía más los impuestos sobre los productos y menos los subsidios no incluidos en el valor de los productos. Se calcula sin hacer deducciones por depreciación de activos fabricados o por agotamiento y degradación de recursos naturales. Los datos están en dólares estadounidenses actuales.
 * `GDP_per_capitaPPP_media`: (`float`) Media histórica del indicador desde 1990 hasta 2023:
   * **Nombre del indicador en inglés**: *GDP per capita, PPP (current international $)*.
   * **Código del indicador**: NY.GDP.PCAP.PP.CD
   * **Descripción**: Este indicador proporciona valores per cápita del producto interno bruto (PIB) expresado en dólares internacionales corrientes convertidos mediante el factor de conversión de paridad de poder adquisitivo (PPA). Es más adecuado para comparar el poder adquisitivo y el nivel de vida entre países, ajustando las diferencias en los costos de vida y los precios.
 * `population_media`: (`float`) Media histórica del indicador desde 1960 hasta 2023:
   * **Nombre del indicador en inglés**: *Population, total*.
   * **Código del indicador**: SP.POP.TOTL
   * **Descripción**: La población total se basa en la definición de facto de población, que cuenta a todos los residentes independientemente de su estatus legal o ciudadanía. Los valores mostrados son estimaciones de mitad de año.
 * `surface_area_media`: (`float`) Media histórica del indicador desde 1961 hasta 2021:
   * **Nombre del indicador en inglés**: *Surface area (sq. km)*.
   * **Código del indicador**: AG.SRF.TOTL.K2
   * **Descripción**: Por superficie se entiende la superficie total de un país, incluidas las zonas cubiertas por masas de agua interiores y algunas vías navegables costeras. Se mide en $km^2$ (kilómetros cuadrados).

Y dimensiones,

In [30]:
df_countries.shape

(278, 20)

Y ya estará listo para ser exportando en el archivo CSV correspondiente

In [31]:
df_countries.to_csv(os.path.join(data_path, "info_countries.csv"), index=False, encoding="utf-8")

### Actualizando dataframe de Medallas

A modo de paso extra, vamos a agregar algunas columnas del dataframe `df_countries` a `df`, que es aquel que tiene la información de las medallas. Quizás este paso no parece necesario, pero más adelante mostrará su relevancia. Haremos Left Join, con `df` situado a la izquierda, usando las columnas referentes al código COI,

In [32]:
df = df.merge(df_countries[["ISO_code", "COI", "Continente"]], how="left", left_on="Country_code_coi", right_on="COI").drop(columns=["COI"])
df = df[['Country', 'Country_name','Country_code_coi', "ISO_code", "Continente", 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']]
df = df.drop_duplicates()
df.sample(5)

,Country,Country_name,Country_code_coi,ISO_code,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
106,Poland (POL),Poland,POL,POL,Europa,23,73,93,142,308,24,7,7,9,23,47,80,100,151,331
109,Qatar (QAT),Qatar,QAT,QAT,Asia,11,2,2,5,9,0,0,0,0,0,11,2,2,5,9
124,Serbia and Montenegro (SCG),Serbia and Montenegro,SCG,NaN,NaN,3,2,4,3,9,3,0,0,0,0,6,2,4,3,9
97,Niger (NIG),Niger,NIG,NER,África,14,0,1,1,2,0,0,0,0,0,14,0,1,1,2
108,Puerto Rico (PUR),Puerto Rico,PUR,PRI,América,20,2,2,8,12,8,0,0,0,0,28,2,2,8,12


Y el dataframe resultante queda con las siguientes dimensiones,

In [33]:
df.shape

(162, 20)

Como se puede observar, se han agregado las columnas referentes al código ISO y al Continente. El código ISO se agregó ya que se considera un dato fundamental al hablar de países que enriquecería los datos del dataframe de medallas. La razón para agregar el Continente se explicará en lo sucesivo.

#### Equipos olímpicos sin continente asignado

Visualicemos aquellos equipos olímpicos con valores nulos, ya sea en el código ISO o en el Continente,

In [34]:
df[df.Continente.isna() | df["ISO_code"].isna()]

,Country,Country_name,Country_code_coi,ISO_code,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
5,Australasia (ANZ),Australasia,ANZ,NaN,NaN,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12
15,Bohemia (BOH),Bohemia,BOH,NaN,NaN,3,0,1,3,4,0,0,0,0,0,3,0,1,3,4
18,British West Indies (BWI),British West Indies,BWI,NaN,NaN,1,0,0,2,2,0,0,0,0,0,1,0,0,2,2
34,Czechoslovakia (TCH),Czechoslovakia,TCH,NaN,NaN,16,49,49,45,143,16,2,8,15,25,32,51,57,60,168
50,United Team of Germany (EUA),United Team of Germany,EUA,NaN,NaN,3,28,54,36,118,3,8,6,5,19,6,36,60,41,137
51,East Germany (GDR),East Germany,GDR,NaN,NaN,5,153,129,127,409,6,39,36,35,110,11,192,165,162,519
52,West Germany (FRG),West Germany,FRG,NaN,NaN,5,56,67,81,204,6,11,15,13,39,11,67,82,94,243
75,Kosovo (KOS),Kosovo,KOS,XKX,NaN,3,3,1,1,5,2,0,0,0,0,5,3,1,1,5
95,Netherlands Antilles (AHO),Netherlands Antilles,AHO,NaN,NaN,13,0,1,0,1,2,0,0,0,0,15,0,1,0,1
110,Refugee Olympic Team (EOR),Refugee Olympic Team,EOR,NaN,NaN,3,0,0,1,1,0,0,0,0,0,3,0,0,1,1


Algunos registros no pudieron ser clasificados en un continente, y es primeramente porque no se encontraban tabulados en el articulo de Wikipedia usado. Sin embargo, las razones por las cuales no se listaron pueden ser variadas:
 1. No son países, sino equipos olímpicos: Australasia, United Team of Germany, Refugee Olympic Team, Unified Team, Olympic Athletes from Russia, ROC, Individual Neutral Athletes, Independent Olympic Athletes, Independent Olympic Participants y Mixed team.
 2. Son países con reconocimiento limitado: Kosovo.
 3. Países o territorio que ya no existen: Bohemia, British West Indies, Czechoslovakia, East Germany, West Germany, Netherlands Antilles, Russian Empire, Soviet Union, Serbia and Montenegro, Yugoslavia.
 4. Territorio dependientes que participan de forma separada a su país: Virgin Islands.

Sin embargo, con un poco de investigación, es posible asignar un continente a alguno de los registros. Ya sea que hayan sido países existentes, hayan representado una región en particular, o sean equipos estrechamente relacionados con uno o varios países de una misma región. A continuación, se crea un mapeo con el continente asignado,

In [35]:
map_paises_sin_continente = {
	'Australasia' : "Oceanía", # Dado que fue un equipo conformado por atletas tanto de Australia como de Nueva Zelanda
	'Bohemia' : "Europa", # Dada su posición geográfica: Ocupó el actual territorio de la República Checa
	'British West Indies' : "América", # Aunque fueron británicas, su posición geográfica y cultural la hace más cercana a América
	'Czechoslovakia' : "Europa", # Por su posición geográfica
	'United Team of Germany' : "Europa", # Los atletas eran oriundos ya sea de East Germany o de West Germany
	'East Germany' : "Europa", # Por posición geográfica
	'West Germany' : "Europa", # Por posición geográfica
	'Kosovo' : "Europa", # Por posición geográfica
	'Netherlands Antilles' : "América", # Por posición geográfica
	'Refugee Olympic Team': np.nan, # Sus atletas son de lugares variados
	'Serbia and Montenegro' : "Europa", # Por posición geográfica
	'Virgin Islands' : "América", # Por posición geográfica
	'Yugoslavia' : "Europa", # Por posición geográfica
	'Independent Olympic Athletes' : np.nan, # Sus atletas son de lugares variados
	'Independent Olympic Participants' : np.nan, # Sus atletas son de lugares variados
	'Mixed team' : np.nan,  # Sus atletas son de lugares variados
	'Russian Empire' : "Europa", # Equipos relacionados con Rusia
	'Soviet Union' : "Europa",
	'Unified Team' : "Europa",
	'Olympic Athletes from Russia' : "Europa",
	'ROC' : "Europa",
	'Individual Neutral Athletes' : "Europa"
}

Dado que anteriormente se ha fijado a Rusia en Europa por razones políticas, los equipos que han sido formados por atletas rusos también serán clasificados de esa forma. 

En fin, he aquí la razón por la cual se agregó la columna `Continente` al dataframe, para poder hacer estás asignaciones. Hacemos las asignaciones con la siguiente linea

In [36]:
df["Continente"] = df.apply(lambda x: map_paises_sin_continente[x.Country_name] if x.Country_name in map_paises_sin_continente else x["Continente"], axis=1)

Y el dataframe final es el siguiente,

In [37]:
df.head()

,Country,Country_name,Country_code_coi,ISO_code,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
0,Afghanistan (AFG),Afghanistan,AFG,AFG,Asia,16,0,0,2,2,0,0,0,0,0,16,0,0,2,2
1,Albania (ALB),Albania,ALB,ALB,Europa,10,0,0,2,2,5,0,0,0,0,15,0,0,2,2
2,Algeria (ALG),Algeria,ALG,DZA,África,15,7,4,9,20,3,0,0,0,0,18,7,4,9,20
3,Argentina (ARG),Argentina,ARG,ARG,América,26,22,27,31,80,20,0,0,0,0,46,22,27,31,80
4,Armenia (ARM),Armenia,ARM,ARM,Europa,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22


Donde los equipos con código ISO nulo son

In [38]:
df[df["ISO_code"].isna()]["Country_name"].values

array(['Australasia', 'Bohemia', 'British West Indies', 'Czechoslovakia',
       'United Team of Germany', 'East Germany', 'West Germany',
       'Netherlands Antilles', 'Refugee Olympic Team', 'Russian Empire',
       'Soviet Union', 'Unified Team', 'Olympic Athletes from Russia',
       'ROC', 'Serbia and Montenegro', 'Yugoslavia',
       'Individual Neutral Athletes', 'Independent Olympic Athletes',
       'Independent Olympic Participants', 'Mixed team'], dtype=object)

Algunos en otrora países tuvieron en su momento un código ISO que perdieron al ser disueltos. A continuación muestro aquellos países con su respectivo código ISO 3166-1 alpha 3:
 * Czechoslovakia: CSK
 * East Germany: DDR
 * West Germany: DEU (mantenido por la Alemania unificada)
 * Netherlands Antilles: ANT
 * Soviet Union: SUN
 * Serbia and Montenegro: SCG
 * Yugoslavia: YUG

No obstante, ninguno de estos países aparecen en `df_countries`, por lo que no podrán participar en los análisis que involucren los indicadores de países. Por ende, no es totalmente necesario agregar estos datos al dataframe de medallas, sin olvidar que el código DEU entraría en conflicto. 

### Exportando el dataframe de Medallas

In [39]:
df.to_csv(os.path.join(data_path,"olympics.csv"), index=False, encoding="utf-8")
df.to_excel(os.path.join(data_path,"olympics_excel.xlsx"), index=False, encoding="utf-8")